In [8]:
import pandas as pd
import re
import json

# Load the data
print("Loading data...")
df = pd.read_csv('medicinal_plants.csv', encoding='utf-8')
print(f"✅ Loaded {len(df)} rows and {len(df.columns)} columns")
print(f"Columns: {list(df.columns)}\n")

# 1. Clean column names
print("1. Cleaning column names...")
df.columns = df.columns.str.strip().str.replace('_', ' ').str.title()
print(f"✅ Cleaned columns: {list(df.columns)}\n")

# 2. Clean text in all columns
print("2. Cleaning text fields...")
text_columns = df.select_dtypes(include=['object']).columns

for col in text_columns:
    # Convert to string and strip whitespace
    df[col] = df[col].astype(str).str.strip()
    # Replace multiple spaces with single space
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
    # Replace 'nan' with empty string
    df[col] = df[col].replace('nan', '')
    df[col] = df[col].replace('NaN', '')

print("✅ Text cleaned\n")

# 3. Split common names properly
print("3. Fixing common names...")
if 'Common Names' in df.columns:
    # Split names that are concatenated (separated by 2+ spaces)
    df['Common Names'] = df['Common Names'].apply(
        lambda x: ', '.join([name.strip() for name in re.split(r'\s{2,}', str(x)) if name.strip()])
    )
    print("✅ Common names separated\n")

# 4. Remove duplicates
print("4. Removing duplicates...")
before = len(df)
df = df.drop_duplicates(subset=['Scientific Name'], keep='first')
after = len(df)
print(f"✅ Removed {before - after} duplicates\n")

# 5. Replace empty strings with None
print("5. Handling missing values...")
df = df.replace('', None)
df = df.replace('N/A', None)

missing = df.isnull().sum()
print("Missing values per column:")
for col, count in missing.items():
    if count > 0:
        print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")
print()

# 6. Add data quality flags
print("6. Adding quality scores...")
df['Has_Description'] = df['Description'].notna() & (df['Description'].str.len() > 50)
df['Has_Uses'] = df['Uses And Properties'].notna() & (df['Uses And Properties'].str.len() > 20)
df['Has_Active_Ingredients'] = df['Active Ingredients'].notna() & (df['Active Ingredients'].str.len() > 20)
df['Has_Family'] = df['Family'].notna() & (df['Family'].str.len() > 0)

# Calculate completeness score
df['Completeness_Score'] = (
    df[['Has_Description', 'Has_Uses', 'Has_Active_Ingredients', 'Has_Family']].sum(axis=1) / 4 * 100
).round(0)

print("✅ Quality scores calculated\n")

# 7. Export cleaned data
print("7. Exporting cleaned data...")

# Full dataset
df.to_csv('medicinal_plants_cleaned.csv', index=False, encoding='utf-8')
print("✅ Saved: medicinal_plants_cleaned.csv")

# High quality only (>50% complete)
high_quality = df[df['Completeness_Score'] > 50]
high_quality.to_csv('medicinal_plants_high_quality.csv', index=False, encoding='utf-8')
print(f"✅ Saved: medicinal_plants_high_quality.csv ({len(high_quality)} plants)")

# Very high quality (>75% complete)
very_high_quality = df[df['Completeness_Score'] > 75]
very_high_quality.to_csv('medicinal_plants_very_high_quality.csv', index=False, encoding='utf-8')
print(f"✅ Saved: medicinal_plants_very_high_quality.csv ({len(very_high_quality)} plants)\n")

# 8. Generate summary report
print("="*70)
print("CLEANING REPORT")
print("="*70)

print(f"\n📊 Dataset Overview:")
print(f"  Total plants: {len(df)}")
print(f"  Unique families: {df['Family'].nunique()}")
print(f"  Columns: {len(df.columns)}")

print(f"\n✅ Data Quality:")
print(f"  Plants with Family: {df['Has_Family'].sum()} ({df['Has_Family'].sum()/len(df)*100:.1f}%)")
print(f"  Plants with Description: {df['Has_Description'].sum()} ({df['Has_Description'].sum()/len(df)*100:.1f}%)")
print(f"  Plants with Uses: {df['Has_Uses'].sum()} ({df['Has_Uses'].sum()/len(df)*100:.1f}%)")
print(f"  Plants with Active Ingredients: {df['Has_Active_Ingredients'].sum()} ({df['Has_Active_Ingredients'].sum()/len(df)*100:.1f}%)")
print(f"  Average Completeness: {df['Completeness_Score'].mean():.1f}%")

print(f"\n📈 Quality Distribution:")
print(f"  >75% complete: {len(very_high_quality)} plants")
print(f"  >50% complete: {len(high_quality)} plants")
print(f"  <50% complete: {len(df[df['Completeness_Score'] <= 50])} plants")

print(f"\n🏆 Top 10 Most Complete Entries:")
top_10 = df.nlargest(10, 'Completeness_Score')[['Scientific Name', 'Common Names', 'Family', 'Completeness_Score']]
for idx, row in top_10.iterrows():
    names = row['Common Names'][:50] if row['Common Names'] else 'N/A'
    print(f"  {row['Scientific Name']:30} | {names:30} | {row['Completeness_Score']:.0f}%")

print(f"\n⚠️  Bottom 10 Least Complete Entries:")
bottom_10 = df.nsmallest(10, 'Completeness_Score')[['Scientific Name', 'Common Names', 'Completeness_Score']]
for idx, row in bottom_10.iterrows():
    names = row['Common Names'][:50] if row['Common Names'] else 'N/A'
    print(f"  {row['Scientific Name']:30} | {names:30} | {row['Completeness_Score']:.0f}%")

# Save summary to JSON
summary = {
    'total_plants': len(df),
    'plants_with_family': int(df['Has_Family'].sum()),
    'plants_with_description': int(df['Has_Description'].sum()),
    'plants_with_uses': int(df['Has_Uses'].sum()),
    'plants_with_active_ingredients': int(df['Has_Active_Ingredients'].sum()),
    'average_completeness': float(df['Completeness_Score'].mean()),
    'high_quality_plants': len(high_quality),
    'very_high_quality_plants': len(very_high_quality)
}

with open('cleaning_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("\n✅ Summary saved to: cleaning_summary.json")
print("\n" + "="*70)
print("✅ CLEANING COMPLETE!")
print("="*70)

print("\n💡 Files created:")
print("  1. medicinal_plants_cleaned.csv - Full cleaned dataset")
print("  2. medicinal_plants_high_quality.csv - Plants >50% complete")
print("  3. medicinal_plants_very_high_quality.csv - Plants >75% complete")
print("  4. cleaning_summary.json - Statistics summary")

# Show sample of cleaned data
print("\n📋 Sample of cleaned data:")
print(df[['Scientific Name', 'Common Names', 'Family', 'Completeness_Score']].head(10))

Loading data...
✅ Loaded 310 rows and 15 columns
Columns: ['scientific_name', 'common_names', 'family', 'description', 'parts_used', 'therapeutic_category', 'uses_and_properties', 'active_ingredients', 'pharmacological_effects', 'notes', 'warning', 'status', 'alternative_names', 'origin', 'dosage']

1. Cleaning column names...
✅ Cleaned columns: ['Scientific Name', 'Common Names', 'Family', 'Description', 'Parts Used', 'Therapeutic Category', 'Uses And Properties', 'Active Ingredients', 'Pharmacological Effects', 'Notes', 'Warning', 'Status', 'Alternative Names', 'Origin', 'Dosage']

2. Cleaning text fields...
✅ Text cleaned

3. Fixing common names...
✅ Common names separated

4. Removing duplicates...
✅ Removed 0 duplicates

5. Handling missing values...
Missing values per column:
  Family: 306 (98.7%)
  Parts Used: 3 (1.0%)
  Therapeutic Category: 8 (2.6%)
  Uses And Properties: 130 (41.9%)
  Active Ingredients: 164 (52.9%)
  Pharmacological Effects: 137 (44.2%)
  Notes: 238 (76.8%)


In [15]:
import pandas as pd
import re
import numpy as np

def clean_medicinal_plants_dataset(df):
    """
    Main function to clean and reorganize the medicinal plants dataset
    """
    print("Starting data cleaning process...")
    
    # Create a new cleaned dataframe
    cleaned_data = []
    
    for idx, row in df.iterrows():
        if pd.isna(row['Scientific Name']) or row['Scientific Name'] == '':
            continue
            
        plant_data = {
            'Scientific_Name': clean_text(row['Scientific Name']),
            'Common_Names': clean_text(row['Common Names']),
            'Family': clean_text(row['Family']),
            'Description': extract_description(row),
            'Parts_Used': extract_parts_used(row),
            'Therapeutic_Category': extract_therapeutic_category(row),
            'Traditional_Uses': extract_traditional_uses(row),
            'Active_Ingredients': extract_active_ingredients(row),
            'Pharmacological_Effects': extract_pharmacological_effects(row),
            'Dosage_Instructions': extract_dosage_instructions(row),
            'Warnings': extract_warnings(row),
            'Origin_Distribution': extract_origin_distribution(row),
            'Cultivation_Status': extract_cultivation_status(row),
            'Preparation_Methods': extract_preparation_methods(row),
            'Clinical_Status': extract_clinical_status(row),
            'Alternative_Names': clean_text(row['Alternative Names']),
            'Additional_Notes': extract_additional_notes(row),
            'Has_Description': row.get('Has_Description', ''),
            'Has_Uses': row.get('Has_Uses', ''),
            'Has_Active_Ingredients': row.get('Has_Active_Ingredients', ''),
            'Has_Family': row.get('Has_Family', ''),
            'Completeness_Score': row.get('Completeness_Score', '')
        }
        cleaned_data.append(plant_data)
    
    cleaned_df = pd.DataFrame(cleaned_data)
    print(f"Cleaning complete. Processed {len(cleaned_df)} plants.")
    return cleaned_df

def clean_text(text):
    """Clean and standardize text"""
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_description(row):
    """Extract plant description"""
    desc = clean_text(row.get('Description', ''))
    if not desc or desc in ['', 'nan']:
        return ""
    
    # Remove content that belongs to other sections
    stop_patterns = [
        r'active ingredients.*',
        r'pharmacological effects.*',
        r'origin.*',
        r'parts used.*',
        r'therapeutic category.*',
        r'uses and properties.*'
    ]
    
    for pattern in stop_patterns:
        desc = re.split(pattern, desc, flags=re.IGNORECASE)[0]
    
    return desc.strip()

def extract_parts_used(row):
    """Extract parts used information"""
    sources = [
        clean_text(row.get('Parts Used', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Uses And Properties', ''))
    ]
    
    for source in sources:
        if 'parts used' in source.lower() or 'parts useD' in source.lower():
            content = extract_after_keyword(source, 'parts used')
            if content:
                return content
    
    # Fallback to Parts Used column
    parts_used = clean_text(row.get('Parts Used', ''))
    if parts_used and parts_used not in ['', 'nan']:
        return parts_used
    
    return ""

def extract_therapeutic_category(row):
    """Extract therapeutic category"""
    sources = [
        clean_text(row.get('Therapeutic Category', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Uses And Properties', ''))
    ]
    
    for source in sources:
        if 'therapeutic category' in source.lower():
            content = extract_after_keyword(source, 'therapeutic category')
            if content:
                return content
    
    # Fallback to Therapeutic Category column
    category = clean_text(row.get('Therapeutic Category', ''))
    if category and category not in ['', 'nan']:
        return category
    
    return ""

def extract_traditional_uses(row):
    """Extract traditional uses and properties"""
    sources = [
        clean_text(row.get('Uses And Properties', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Pharmacological Effects', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if any(keyword in lower_source for keyword in ['uses and properties', 'uses anD properties', 'traditional use', 'used to treat']):
            if 'uses and properties' in lower_source:
                content = extract_after_keyword(source, 'uses and properties')
            elif 'uses anD properties' in lower_source:
                content = extract_after_keyword(source, 'uses anD properties')
            else:
                content = source
            
            if content:
                # Clean the content
                content = remove_section(content, 'pharmacological effects')
                content = remove_section(content, 'active ingredients')
                content = remove_section(content, 'warning')
                return content.strip()
    
    return ""

def extract_active_ingredients(row):
    """Extract active ingredients information"""
    sources = [
        clean_text(row.get('Active Ingredients', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Origin', '')),
        clean_text(row.get('Pharmacological Effects', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if any(keyword in lower_source for keyword in ['active ingredients', 'active ingreDients', 'main compound', 'contains']):
            if 'active ingredients' in lower_source:
                content = extract_after_keyword(source, 'active ingredients')
            elif 'active ingreDients' in lower_source:
                content = extract_after_keyword(source, 'active ingreDients')
            else:
                # Try to extract relevant portion
                content = extract_ingredients_content(source)
            
            if content and len(content) > 10:  # Meaningful content
                return content.strip()
    
    return ""

def extract_pharmacological_effects(row):
    """Extract pharmacological effects"""
    sources = [
        clean_text(row.get('Pharmacological Effects', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Uses And Properties', '')),
        clean_text(row.get('Notes', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if 'pharmacological effects' in lower_source:
            content = extract_after_keyword(source, 'pharmacological effects')
            if content:
                content = remove_section(content, 'uses and properties')
                content = remove_section(content, 'warning')
                content = remove_section(content, 'notes')
                return content.strip()
    
    return ""

def extract_dosage_instructions(row):
    """Extract dosage instructions"""
    sources = [
        clean_text(row.get('Dosage', '')),
        clean_text(row.get('Uses And Properties', '')),
        clean_text(row.get('Notes', '')),
        clean_text(row.get('Description', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if any(keyword in lower_source for keyword in ['dosage', 'preparation and dosage', 'dose']):
            if 'preparation and dosage' in lower_source:
                content = extract_after_keyword(source, 'preparation and dosage')
            elif 'dosage' in lower_source:
                content = extract_after_keyword(source, 'dosage')
            else:
                content = extract_dosage_content(source)
            
            if content:
                return content.strip()
    
    return ""

def extract_warnings(row):
    """Extract warnings and precautions"""
    sources = [
        clean_text(row.get('Warning', '')),
        clean_text(row.get('Notes', '')),
        clean_text(row.get('Uses And Properties', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Pharmacological Effects', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if 'warning' in lower_source:
            content = extract_after_keyword(source, 'warning')
            if content:
                content = remove_section(content, 'notes')
                content = remove_section(content, 'status')
                return content.strip()
    
    return ""

def extract_origin_distribution(row):
    """Extract origin and distribution information"""
    sources = [
        clean_text(row.get('Origin', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Notes', ''))
    ]
    
    for source in sources:
        lower_source = source.lower()
        if 'origin' in lower_source:
            content = extract_after_keyword(source, 'origin')
            if content:
                content = remove_section(content, 'active ingredients')
                content = remove_section(content, 'pharmacological effects')
                content = remove_section(content, 'parts used')
                return content.strip()
    
    return ""

def extract_cultivation_status(row):
    """Extract cultivation status"""
    sources = [
        clean_text(row.get('Origin', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Notes', ''))
    ]
    
    cultivation_keywords = ['cultivated', 'wild', 'harvested', 'cultivation', 'grown', 'plantation']
    
    for source in sources:
        for keyword in cultivation_keywords:
            if keyword in source.lower():
                # Extract sentence or context around the keyword
                sentences = re.split(r'[.!?]', source)
                for sentence in sentences:
                    if keyword in sentence.lower():
                        return sentence.strip()
    
    return ""

def extract_preparation_methods(row):
    """Extract preparation methods"""
    sources = [
        clean_text(row.get('Dosage', '')),
        clean_text(row.get('Uses And Properties', '')),
        clean_text(row.get('Notes', ''))
    ]
    
    preparation_keywords = ['infusion', 'tincture', 'decoction', 'extract', 'tea', 'ointment', 'powder']
    
    methods = []
    for source in sources:
        for keyword in preparation_keywords:
            if keyword in source.lower():
                # Extract relevant sentences
                sentences = re.split(r'[.!?]', source)
                for sentence in sentences:
                    if keyword in sentence.lower():
                        methods.append(sentence.strip())
    
    return ' '.join(methods) if methods else ""

def extract_clinical_status(row):
    """Extract clinical and regulatory status"""
    sources = [
        clean_text(row.get('Status', '')),
        clean_text(row.get('Notes', ''))
    ]
    
    status_keywords = ['clinical', 'pharm', 'traditional', 'who', 'commission', 'escop', 'hmpc']
    
    status_info = []
    for source in sources:
        for keyword in status_keywords:
            if keyword in source.lower():
                # Extract the relevant line or section
                lines = source.split('\n')
                for line in lines:
                    if keyword in line.lower():
                        status_info.append(line.strip())
    
    return ' | '.join(status_info) if status_info else ""

def extract_additional_notes(row):
    """Extract additional notes"""
    sources = [
        clean_text(row.get('Notes', '')),
        clean_text(row.get('Description', '')),
        clean_text(row.get('Uses And Properties', ''))
    ]
    
    notes_keywords = ['notes', 'note:', 'additional', 'also known', 'related species']
    
    for source in sources:
        lower_source = source.lower()
        if 'notes' in lower_source:
            content = extract_after_keyword(source, 'notes')
            if content:
                return content.strip()
    
    # If no specific notes section, use the Notes column
    notes = clean_text(row.get('Notes', ''))
    if notes and notes not in ['', 'nan']:
        return notes
    
    return ""

# Helper functions
def extract_after_keyword(text, keyword):
    """Extract content after a specific keyword"""
    pattern = r'{}(.*)'.format(re.escape(keyword))
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return ""

def remove_section(text, section_name):
    """Remove a specific section from text"""
    pattern = r'{}(.*)'.format(re.escape(section_name))
    return re.split(pattern, text, flags=re.IGNORECASE)[0].strip()

def extract_ingredients_content(text):
    """Extract ingredients-related content"""
    # Look for patterns that indicate ingredient information
    ingredient_indicators = [
        r'contains.*?[.!?]',
        r'main compounds?.*?[.!?]',
        r'active compounds?.*?[.!?]',
        r'ingredients?.*?[.!?]'
    ]
    
    for pattern in ingredient_indicators:
        matches = re.findall(pattern, text, re.IGNORECASE)
        if matches:
            return ' '.join(matches)
    
    return ""

def extract_dosage_content(text):
    """Extract dosage-related content"""
    dosage_indicators = [
        r'dose.*?[.!?]',
        r'dosage.*?[.!?]',
        r'take.*?[0-9].*?[.!?]',
        r'[0-9].*?g.*?[.!?]',
        r'[0-9].*?mg.*?[.!?]',
        r'[0-9].*?ml.*?[.!?]'
    ]
    
    dosage_info = []
    for pattern in dosage_indicators:
        matches = re.findall(pattern, text, re.IGNORECASE)
        dosage_info.extend(matches)
    
    return ' '.join(dosage_info) if dosage_info else ""

def analyze_cleaning_results(cleaned_df, original_df):
    """Analyze the results of the cleaning process"""
    print("\n" + "="*50)
    print("CLEANING RESULTS ANALYSIS")
    print("="*50)
    
    print(f"Original dataset shape: {original_df.shape}")
    print(f"Cleaned dataset shape: {cleaned_df.shape}")
    
    # Check completeness of key columns
    key_columns = ['Description', 'Active_Ingredients', 'Pharmacological_Effects', 'Traditional_Uses']
    for col in key_columns:
        non_empty = cleaned_df[col].apply(lambda x: len(str(x)) > 10).sum()
        percentage = (non_empty / len(cleaned_df)) * 100
        print(f"{col}: {non_empty}/{len(cleaned_df)} ({percentage:.1f}%) non-empty")
    
    # Show sample of cleaned data
    print("\nSAMPLE OF CLEANED DATA (first 3 plants):")
    print("="*50)
    for i in range(min(3, len(cleaned_df))):
        plant = cleaned_df.iloc[i]
        print(f"\n{i+1}. {plant['Scientific_Name']}")
        print(f"   Family: {plant['Family']}")
        print(f"   Active Ingredients: {plant['Active_Ingredients'][:100]}{'...' if len(str(plant['Active_Ingredients'])) > 100 else ''}")
        print(f"   Uses: {plant['Traditional_Uses'][:100]}{'...' if len(str(plant['Traditional_Uses'])) > 100 else ''}")

def export_cleaning_report(cleaned_df, output_file='cleaning_report.txt'):
    """Generate a detailed cleaning report"""
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("MEDICINAL PLANTS DATA CLEANING REPORT\n")
        f.write("=" * 50 + "\n\n")
        
        f.write(f"Total plants processed: {len(cleaned_df)}\n\n")
        
        f.write("COLUMN COMPLETENESS:\n")
        f.write("-" * 30 + "\n")
        for col in cleaned_df.columns:
            non_empty = cleaned_df[col].apply(lambda x: len(str(x)) > 10).sum()
            percentage = (non_empty / len(cleaned_df)) * 100
            f.write(f"{col:25} : {non_empty:3d}/{len(cleaned_df):3d} ({percentage:5.1f}%)\n")

# MAIN EXECUTION - This is what you run
def main():
    """Main function to run the complete cleaning process"""
    
    # Load your Excel file
    try:
        print("Loading medicinal plants dataset...")
        df = pd.read_csv('medicinal_plants_cleaned.csv')
        print(f"Successfully loaded dataset with {len(df)} rows and {len(df.columns)} columns")
        
        # Display original column names for verification
        print("\nOriginal columns:")
        for col in df.columns:
            print(f"  - {col}")
        
    except FileNotFoundError:
        print("Error: File 'medicinal_plants_cleaned.xlsx' not found.")
        print("Please make sure the file is in the same directory as this script.")
        return
    except Exception as e:
        print(f"Error loading file: {e}")
        return
    
    # Clean the dataset
    cleaned_df = clean_medicinal_plants_dataset(df)
    
    # Analyze results
    analyze_cleaning_results(cleaned_df, df)
    
    # Generate report
    export_cleaning_report(cleaned_df)
    print(f"\nCleaning report saved to 'cleaning_report.txt'")
    
    # Save cleaned data
    output_filename = 'medicinal_plants_cleaned_final.xlsx'
    cleaned_df.to_excel(output_filename, index=False)
    print(f"Cleaned dataset saved to '{output_filename}'")
    
    print("\nCleaning process completed successfully!")

# Run the main function
if __name__ == "__main__":
    main()

Loading medicinal plants dataset...
Successfully loaded dataset with 310 rows and 20 columns

Original columns:
  - Scientific Name
  - Common Names
  - Family
  - Description
  - Parts Used
  - Therapeutic Category
  - Uses And Properties
  - Active Ingredients
  - Pharmacological Effects
  - Notes
  - Warning
  - Status
  - Alternative Names
  - Origin
  - Dosage
  - Has_Description
  - Has_Uses
  - Has_Active_Ingredients
  - Has_Family
  - Completeness_Score
Starting data cleaning process...
Cleaning complete. Processed 310 plants.

CLEANING RESULTS ANALYSIS
Original dataset shape: (310, 20)
Cleaned dataset shape: (310, 22)
Description: 310/310 (100.0%) non-empty
Active_Ingredients: 274/310 (88.4%) non-empty
Pharmacological_Effects: 74/310 (23.9%) non-empty
Traditional_Uses: 170/310 (54.8%) non-empty

SAMPLE OF CLEANED DATA (first 3 plants):

1. Abies alba
   Family: Pinaceae
   Active Ingredients: Fir needle oil contains vola- Cones are typically erect and disintegrates at maturity